In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.insert(0, '..')
from libgen.file_io import *
from libgen.search_utils import *
from libgen.spectral_operations import *
from tqdm import tqdm
import os

In [2]:
def extract_peaks(pos_dir, neg_dir, std_list):
    msms = []
    std_list_msms = std_list.copy()
    for index, row in tqdm(std_list.iterrows(), total = len(std_list)):
        if 'neg' in row['File']:
            working_dir = neg_dir
            polarity = 'Negative'
        elif 'pos' in row['File']:
            working_dir = pos_dir
            polarity = 'Positive'
        ms1, ms2 = read_mzml(row['File'], working_dir)

        spectra_temp = quick_search_values(ms2, 'precursor_mz', row['Precursor m/z (Da)']-0.005, row['Precursor m/z (Da)']+0.005)
        spectra_temp = quick_search_values(spectra_temp, 'rt', row['RT [min]']-5/60, row['RT [min]']+5/60)
        if len(spectra_temp) >0:

            
            spectra_temp['adjusted_intensity'] = spectra_temp['precursor_intensity']*spectra_temp['peak_purity']/100
            std_list_msms.loc[index, 'precursor_mz'] = spectra_temp.iloc[0]['precursor_mz']
            spectra_temp.sort_values(by = 'adjusted_intensity', ascending = False, inplace = True)
            std_list_msms.loc[index, 'precursor_intensity'] = spectra_temp.iloc[0]['precursor_intensity']
            
            std_list_msms.loc[index, 'IONMODE'] = polarity
            msms.append(spectra_temp.iloc[0]['peaks'])
            std_list_msms.loc[index, 'found']=True
        else:
            std_list_msms.loc[index, 'found']=False
            msms.append(np.nan)
        # break
    std_list_msms['peaks']=msms
    std_list_msms.rename(columns = {'Precursor m/z (Da)':'reference_precursor_mz'}, inplace = True)
    return std_list_msms
def write_to_msp_sirius(df, file_path, msms_col = 'peaks', normalize = False):
    if normalize == True:
        df[msms_col] = [normalize_spectrum(peak) for peak in df[msms_col]]
    with open(file_path, 'w') as f:
        for _, row in df.iterrows():
            # Write the name of the spectrum
            f.write(f"NAME: Compound_{row['Compound Number']}\n")
            f.write(f"PRECURSORMZ: {row['precursor_mz']:.4f}\n")
            f.write(f"RETENTIONTIME: {row['RT [min]']}\n")
            f.write(f"IONMODE: {row['IONMODE']}\n")
            f.write(f"SPECTRUMTYPE: {row['Priority/Bonus']}\n")
            f.write(f"COMMENTS: Precursor intensity is {row['precursor_intensity']}\n")
            
            # Write the peaks (assuming each peak is a tuple of (m/z, intensity))
            f.write(f"Num Peaks: {len(row[msms_col])}\n")
            for mz, intensity in row[msms_col]:
                f.write(f"{mz} {intensity}\n")
            
            # Separate spectra by an empty line
            f.write("\n")

# lets try priority first shall we?

In [3]:
pos_dir = '/Users/fanzhoukong/Documents/GitHub/Libgen_data/casmi/1_Priority - Challenges 1-250/pos'
neg_dir = '/Users/fanzhoukong/Documents/GitHub/Libgen_data/casmi/1_Priority - Challenges 1-250/neg'
std_list = pd.read_csv('/Users/fanzhoukong/Documents/GitHub/Libgen_data/casmi/MetSoc2022_CASMI_Workshop_Challenges_Priority_1-250_040522_Share.xlsx - SHARE_PRIORITY.csv')

In [4]:
std_list_msms_pri = extract_peaks(pos_dir, neg_dir, std_list)

100%|██████████| 250/250 [22:52<00:00,  5.49s/it]


In [30]:
write_to_msp_sirius(std_list_msms, '/Users/fanzhoukong/Documents/GitHub/Libgen_data/casmi/1_Priority - Challenges 1-250/priority.msp', normalize = False)

# lets then do bonus

In [10]:
master_dir = '/Users/fanzhoukong/Documents/GitHub/Libgen_data/casmi/2_Bonus - Challenges 251-500'
pos_dir = os.path.join(master_dir, 'pos')
neg_dir = os.path.join(master_dir, 'neg')
std_list_bouns = pd.read_csv('/Users/fanzhoukong/Documents/GitHub/Libgen_data/casmi/MetSoc2022_CASMI_Workshop_Challenges_Bonus_251-500_040522_Share.xlsx - SHARE_BONUS.csv')

In [11]:
std_list_bouns

,Compound Number,File,RT [min],Precursor m/z (Da),Priority/Bonus
0,251,A_M22_negPFP_03,1.03,209.0819,Bonus
1,252,A_M15_negPFP_03,0.79,359.1348,Bonus
2,253,A_M12_posPFP_01,3.22,455.1489,Bonus
3,254,A_M15_posPFP_01,3.69,329.2323,Bonus
4,255,A_M1_posPFP_01,1.72,265.1071,Bonus
...,...,...,...,...,...
245,496,A_M15_negPFP_03,2.56,279.0874,Bonus
246,497,A_M19_negPFP_03,4.42,971.4857,Bonus
247,498,A_M13_negPFP_03,5.65,345.0616,Bonus
248,499,A_M2_negPFP_03,3.41,717.1461,Bonus


In [6]:
from libgen.spectra_plotter import *

In [7]:
std_list_msms_bon = extract_peaks(pos_dir, neg_dir, std_list)

  0%|          | 0/250 [00:00<?, ?it/s]

100%|██████████| 250/250 [07:05<00:00,  1.70s/it]


In [13]:
std_list_msms_all = pd.concat([std_list_msms_pri, std_list_msms_bon], axis = 0)

In [17]:
std_list_msms_all=std_list_msms_all[std_list_msms_all['found']==True]

In [18]:
save_df(std_list_msms_all, '/Users/fanzhoukong/Documents/GitHub/Libgen_data/casmi/all_msms.csv')

['peaks']


100%|██████████| 499/499 [00:00<00:00, 10764.31it/s]


In [21]:
chemcial_abundant = [0.1,0.2]
electric_abundant = [2,10]
formula_db = pd.read_csv('/Users/fanzhoukong/Documents/GitHub/Libgen_data/formula_db/formulaDB_sorted.csv')

In [30]:
from libgen.noise import generate_chemical_noise, generate_noise, add_noise

In [55]:
chemical_lamda = 50
electric_lamda = 5
def mixing_noise(data_raw, chemical_noise_level, electronic_noise_level):
    data = data_raw.copy()
    msms_cont_all = []
    for index, row in tqdm(data.iterrows(),total = len(data)):
        if row['IONMODE']=='negative':
            polarity = '-'
        else:
            polarity = '+'
        noise_chemical = generate_chemical_noise(row['reference_precursor_mz'],chemical_lamda, polarity, formula_db=formula_db, n = np.int64(np.ceil(chemical_noise_level*len(row['peaks']))))
        noise_electric = generate_noise(row['reference_precursor_mz'], electric_lamda, n = np.int64(np.ceil(electronic_noise_level*len(row['peaks']))))
        msms_cont = add_noise(row['peaks'], noise_chemical)
        # print(len(row['peaks']))
        
        msms_cont = add_noise(msms_cont, noise_electric)
        msms_cont_all.append(msms_cont)
    data['peaks_cont'] = msms_cont_all
    return data
        

In [56]:
data_cont_mid = mixing_noise(std_list_msms_all,0.2, 2)
data_cont_low = mixing_noise(std_list_msms_all,0.1, 1)

100%|██████████| 499/499 [00:09<00:00, 51.80it/s]


In [57]:
save_df(data_cont_mid, '/Users/fanzhoukong/Documents/GitHub/Libgen_data/casmi/all_msms_mid_cont.csv')
save_df(data_cont_low, '/Users/fanzhoukong/Documents/GitHub/Libgen_data/casmi/all_msms_low_cont.csv')

['peaks', 'peaks_cont']


100%|██████████| 499/499 [00:00<00:00, 5063.48it/s]


['peaks', 'peaks_cont']


100%|██████████| 499/499 [00:00<00:00, 7367.05it/s]


In [58]:
write_to_msp_sirius(std_list_msms_all, '/Users/fanzhoukong/Documents/GitHub/Libgen_data/casmi/all.msp', normalize = False)
write_to_msp_sirius(data_cont_mid, '/Users/fanzhoukong/Documents/GitHub/Libgen_data/casmi/all_mid_cont.msp', normalize = False, msms_col='peaks_cont')
write_to_msp_sirius(data_cont_low, '/Users/fanzhoukong/Documents/GitHub/Libgen_data/casmi/all_low_cont.msp', normalize = False, msms_col='peaks_cont')